In [1]:
from Scripts import loading as dl, cleaning as cl, profiling as pf
from langdetect import detect

In [2]:
input_link = '../Data/reviews_Cell_Phones_and_Accessories_5.json.gz'
df = dl.load_amazon_full(input_link)

In [3]:
df.head()

,text,label
0,They look good and stick good! I just don't li...,4.0
1,These stickers work like the review says they ...,5.0
2,These are awesome and make my phone look so st...,5.0
3,Item arrived in great time and was in perfect ...,4.0
4,"awesome! stays on, and looks great. can be use...",5.0


In [4]:
pf.get_review_count(df)
total = len(df)
df.describe(include='all')

Amount of reviews:  194439


,text,label
count,194439,194439.000000
unique,194186,NaN
top,,NaN
freq,99,NaN
mean,NaN,4.129912
std,NaN,1.222499
min,NaN,1.000000
25%,NaN,4.000000
50%,NaN,5.000000
75%,NaN,5.000000


In [5]:
df_no_text = df[df.label.isnull()]
print(len(df_no_text))

0


In [6]:
# DATA COMPLETENESS
# Del incomplete entries based on missing values
print('Missing review data as percentage: {:.2%} '.format(len(df[df.text.isnull()])/total))
print('Missing rating information as percentage: {:.2%}'.format(len(df[df.text.isnull()&df.label.isnull()])/total))
df = df[df.text.notnull()&df.label.notnull()]
print("After removing missing text and ratings: ", len(df))

Missing review data as percentage: 0.00% 
Missing rating information as percentage: 0.00%
After removing missing text and ratings:  194439


In [7]:
# Del incomplete entries based on implictly missing values
df = df[df.text!='']
print("After removing empty text: ", len(df))

After removing empty text:  194340


In [8]:
# Del incomplete entries based on implictly missing values
# Using Language detection for detecting invalid texts
def detectLang(row):
    try:
        return detect(row['text'])
    except:
        print("exception:", row['text'])
    else:
        print("sad:", row['text'])
df['LANGUAGE'] = df.apply(detectLang, axis=1)
df['LANGUAGE'].value_counts()

exception: &#1576;&#1589;&#1585;&#1575;&#1581;&#1607; &#1575;&#1606;&#1575; &#1605;&#1606; &#1610;&#1608;&#1605; &#1605;&#1575; &#1588;&#1585;&#1610;&#1578; &#1607;&#1584;&#1575; &#1575;&#1604;&#1605;&#1606;&#1578;&#1580; &#1608;&#1575;&#1606;&#1575; &#1587;&#1593;&#1610;&#1583; ... &#1608;&#1605;&#1585;&#1578;&#1575;&#1581; &#1604;&#1575;&#1606; &#1575;&#1604;&#1576;&#1591;&#1575;&#1585;&#1610;&#1607; &#1588;&#1594;&#1575;&#1604;&#1607; &#1593;&#1604;&#1609; &#1575;&#1603;&#1605;&#1604; &#1608;&#1580;&#1607; &#1608;&#1601;&#1593;&#1604;&#1609; &#1578;&#1587;&#1578;&#1575;&#1607;&#1604; &#1602;&#1610;&#1605;&#1578;&#1607;&#1575; &#1608;&#1575;&#1575;&#1610;&#1583; &#1607;&#1584;&#1610; &#1575;&#1604;&#1605;&#1575;&#1585;&#1603;&#1607; &#1604;&#1575;&#1606;&#1610; &#1580;&#1585;&#1576;&#1578; &#1593;&#1583;&#1577; &#1576;&#1591;&#1575;&#1585;&#1610;&#1575;&#1578; &#1579;&#1575;&#1606;&#1608;&#1610;&#1607; &#1608;&#1575;&#1601;&#1590;&#1604; &#1607;&#1584;&#1610; &#1575;&#1604;&#1605;&#1

en    193751
es       120
af        68
so        61
ro        58
de        38
sl        34
ca        21
fr        18
cy        18
pl        16
tl        14
sq        13
no        11
nl        11
it        11
et        10
vi         8
tr         7
da         7
pt         7
sk         6
id         4
hr         4
sw         3
cs         3
sv         3
fi         3
hu         1
Name: LANGUAGE, dtype: int64

In [19]:
df_dup = df[df.duplicated(subset=['text','label'], keep='last')]
df_dup2 = df[df.duplicated(subset=['text'], keep='last')]
pf.create_word_count(df_dup)
df_dup['word_count'].describe()
pf.create_word_count(df_dup2)
df_dup2['word_count'].describe()
print("Duplicate text and label from reviews without missing information: {:.2%}".format(len(df_dup)/ len(df)))
print("Duplicate text: {:.2%}".format(len(df_dup2)/ len(df)))

Duplicate text and label from reviews without missing information: 0.06%
Duplicate text: 0.08%


In [35]:
df= df.drop_duplicates(subset=['text','label'], keep='last')
df = df.drop_duplicates(subset=['text'], keep='last')
print("After removing duplicate entries and texts: ", len(df))

After removing duplicate entries and texts:  194185


In [23]:
pf.create_word_count(df)

Empty DataFrame
Columns: [text, label, LANGUAGE, word_count]
Index: []


In [45]:
# Del what is not recognized as english
df = df[df['LANGUAGE'] == "en"]
print("After removing non-english text:", len(df))

After removing non-english text: 193678


In [59]:
# Other heuristics: testing the exceptions from langdetect
# Test on invalid
# df[df.text.str.contains('&#1575;')]   # -> del 6536
# df[df.text.str.contains('&#1576;')]   # 6536
# df[df.text=='#2']
# df[df.text.str.contains('&#1588')]    # 6536
# df[df.text.str.contains('&#20294;')]
# df[df.text.str.contains('&#1603;')]   # 6536
df=df[~df.text.str.contains('&#1575;')]
len(df)

193677

In [61]:
df.to_pickle('../Data/amazon_phone.pkl')

